In [1]:
import pandas as pd
import plotly
import numpy as np
import plotly.graph_objs as go
from plotly.offline import iplot
from string import ascii_letters
plotly.offline.init_notebook_mode(connected=True)
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('./q_a_users.csv', low_memory=False)
badges_conv = pd.read_csv('./badges_converts_relevant.csv', low_memory=False)
badges_q = pd.read_csv('./badges_all_pure_q.csv', low_memory=False)
comments = pd.read_csv('./comments.csv', low_memory=False)

In [3]:
badges = pd.concat([badges_conv, badges_q], sort=False)

In [4]:
badges_num = badges.groupby('user_id').agg('count')['name'].to_frame().rename(columns = {'name': 'num_badges'})

In [5]:
comments.drop('Unnamed: 0', axis=1, inplace=True)
comments.rename(columns={'f0_': 'avg_comment_score'}, inplace=True)

In [6]:
df.rename(columns={'owner_antw': 'user_id'}, inplace=True)
df_c = df.join(comments.set_index('user_id'), on='user_id')
df_cb = df_c.join(badges_num, on='user_id')

In [7]:
df_cb.fillna(0, inplace=True)

In [8]:
df_cb.drop('Unnamed: 0', axis=1, inplace=True) # drop redundant id column

# Model

In [10]:
import sklearn.model_selection
import sklearn.metrics
import sklearn.tree

In [12]:
df_cb['min_date_antw'] = pd.to_datetime(df_cb['min_date_antw'])
df_cb['min_date_frage'] = pd.to_datetime(df_cb['min_date_frage'])
df_cb = df_cb[df_cb['min_date_frage'] > pd.to_datetime(0)]
df_cb['target'] = [np.sign(x.day-y.day) if x != pd.to_datetime(0) else -2 for x,y in zip(df_cb['min_date_antw'], df_cb['min_date_frage'])]
model_df = df_cb[
    ['anzahl_fragen', 'avg_score_frage', 'anzahl_comments', 'avg_comment_score', 'num_badges', 'target']
].sample(frac=1.0).copy()
#model_df.dropna(inplace=True)

In [13]:
train, test = sklearn.model_selection.train_test_split(model_df)

In [14]:
clf = sklearn.tree.DecisionTreeClassifier()
clf.fit(train.drop('target', axis=1), train['target'])
clf.score(test.drop('target', axis=1), test['target'])

0.7972165828110748

In [15]:
print(sklearn.metrics.classification_report(test['target'], clf.predict(test.drop('target', axis=1))))

             precision    recall  f1-score   support

         -2       1.00      1.00      1.00    489300
         -1       0.41      0.30      0.34    110155
          0       0.12      0.05      0.07     36729
          1       0.50      0.68      0.58    145517

avg / total       0.78      0.80      0.79    781701



In [21]:
dot_data = sklearn.tree.export_graphviz(clf, out_file='tree.dot')